In [2]:
import openai
import json
import numpy as np
from numpy.linalg import norm
import requests
import csv
api_key = open("api_key").read()
openai.api_key = api_key

In [3]:
def get_embedding(text, model="text-embedding-ada-002"):
   # text = text.replace("\n", " ")
   # return openai.Embedding.create(input = [text], model=model)['data'][0]['embedding']
   url = 'https://api.openai.com/v1/embeddings'
   headers = {
      'Content-Type': 'application/json',
      'Authorization': "Bearer {}".format(api_key)
   }
   data = {
      "input": text,
      "model": model
   }
   res = requests.post(url, headers=headers, json=data)
   res = res.json()
   return res['data'][0]['embedding']

def cos_sim(a, b):
   return np.dot(a, b)/(norm(a)*norm(b))

def save_json(data, filepath=r'new_data.json'):
   with open(filepath, 'w') as fp:
      json.dump(data, fp, indent=4)

In [ ]:
def merge_paragraphs(sentences):
    sentence_list = [" ".join(sentence_word_list) for sentence_word_list in sentences] # merge the words into sentences
    paragraph = " ".join(sentence_list)
    return paragraph

In [ ]:
# All_articles = json.load(open("data/raw/AllTheNews/events/2016_10p.json"))
All_articles = json.load(open("data/result/AllTheNews/network/articles.json"))
existing_embeddings = json.load(open("data/raw/AllTheNews/embeddings/2016_10p.json"))

In [ ]:
vis_articles = csv.DictReader(open('data/raw/VisPub/IEEE VIS papers 1990-2022 - Main dataset-2.csv'))
res = []
count = -1
for article in vis_articles:
    count += 1
    article['id'] = 'vis_{}'.format(count)
    res.append(article)
save_json(res, 'data/raw/VisPub/articles.json')

In [ ]:
# Vis Pub
# vis_articles = csv.DictReader(open('data/raw/VisPub/IEEE VIS papers 1990-2022 - Main dataset-2.csv'))
vis_articles = json.load(open('data/raw/VisPub/articles_w_keywords.json'))
# existing_embeddings = json.load(open("data/raw/VisPub/keyword_embeddings/keyword_embeddings.json"))
existing_ids = [embedding['keyword'] for embedding in existing_embeddings]
# existing_embeddings = []
# existing_ids = []
count = -1
for article in vis_articles:
    count += 1
    print("{}/{}".format(count, 3620))
    # if 'vis_{}'.format(count) in existing_ids: 
    #     continue
    try:
        article['embedding'] = get_embedding(article['Abstract'])
        existing_embeddings.append({
            'doc_id': 'vis_{}'.format(count),
            'title': article['Title'],
            'summary': article['Abstract'],
            'content': article['Abstract'],
            'embedding': article['embedding']
        })
    except:
        print(count, article['Abstract'])
        continue
save_json(existing_embeddings, 'data/result/VisPub/embeddings.json')
print(len(existing_embeddings), count)


In [ ]:
embeddings = []
count = 0
for article in vis_articles:
    embeddings.append({
        'doc_id': 'vis_{}'.format(count),
        'title': article['Title'],
        'summary': article['Abstract'],
        'content': article['Abstract'],
        'embedding': article['embedding']
    })
    count += 1
save_json(embeddings, 'data/result/VisPub/embeddings.json')

In [ ]:
all_article_ids = list(All_articles.keys())
existing_article_ids = [article['id'] for article in existing_embeddings]
target_article_ids = list(set(all_article_ids) - set(existing_article_ids))
print(len(existing_embeddings), len(all_article_ids), len(target_article_ids))

In [ ]:
# All The News
for article_id in target_article_ids:
    article = All_articles[article_id]
    article['embedding'] = get_embedding(article['summary'])
    existing_embeddings.append({
        "id": article_id,
        "summary": article['summary'],
        "content": article['content'],
        "embedding": article['embedding'],
    })
    print("add: ", len(existing_embeddings))
print(len(existing_embeddings))
# save_json(existing_embeddings, "data/raw/AllTheNews/embeddings/2016_10p.json")

In [ ]:
save_json(existing_embeddings, "data/raw/AllTheNews/embeddings/2016_10p.json")

In [ ]:
All_articles = json.load(open("data/raw/AllTheNews/events/2016_10p.json"))
All_article_embeddings = json.load(open("data/raw/AllTheNews/embeddings/2016_10p.json"))
print(len(All_article_embeddings), len(All_articles))

In [ ]:
server_embeddings = json.load(open("data/result/AllTheNews/network/server/embeddings.json"))
embeddings_dict = { doc['doc_id']: doc for doc in server_embeddings}
frontend_data = json.load(open('data/result/AllTheNews/network/server/frontend_2.json'))
article_node_ids = [node['id'] for node in frontend_data['nodes'] if node['type'] == 'article']
for article_id in article_node_ids:
    if article_id not in embeddings_dict:
        print(article_id)
        # embeddings_dict[article_id] = {
        #     'doc_id': article_id,
        #     'embedding': get_embedding(All_articles[article_id]['summary']),
        # }

In [ ]:
entity_dict = json.load(open('data/result/AllTheNews/network/entities_w_description.json'))
res = {}
count = 0
for entity_id, entity_data in entity_dict.items():
    count += 1
    print("{}/{}".format(count, len(entity_dict)))
    try:
        if entity_data['title'] == entity_data['description']:
            content = entity_data['title']
        else:
            content = entity_data['title'] + ", " + entity_data['description']
        embedding = get_embedding(content)
        res[entity_id] = {
            'id': entity_id,
            'content': content,
            'description': entity_data['description'],
            'embedding': embedding,
        }
    except:
        continue
save_json(res, 'data/result/AllTheNews/network/entities_w_description_embedding.json')

In [ ]:
entities_embeddings = json.load(open('data/result/AllTheNews/network/entities_w_description_embedding.json'))
entities = json.load(open('data/result/AllTheNews/network/entities_w_description.json'))
for entity_id, entity_data in entities.items():
    if entity_id not in entities_embeddings:
        if entity_data['title'] == entity_data['description']:
            content = entity_data['title']
        else:
            content = entity_data['title'] + ", " + entity_data['description']
        embedding = get_embedding(content)
        entities_embeddings[entity_id] = {
            'id': entity_id,
            'content': content,
            'description': entity_data['description'],
            'embedding': embedding,
        }
save_json(entities_embeddings, 'data/result/AllTheNews/network/entities_w_description_embedding.json')


In [ ]:
server_embeddings = json.load(open("data/result/AllTheNews/network/server/embeddings.json"))
articles = json.load(open("data/result/AllTheNews/preprocessed/2016_10p.json"))
article_dict = { article['id']: article for article in articles}
for article in server_embeddings:
    article_id = article['doc_id']
    article_title = article_dict[article_id]['title']
    article['title'] = article_title
save_json(server_embeddings, "data/result/AllTheNews/network/server/embeddings_w_title.json")


In [ ]:
embeddings = json.load(open('data/result/VisPub/embeddings.json'))
embeddings_dict = { doc['doc_id']: doc for doc in embeddings}
vis_articles = json.load(open('data/result/VisPub/preprocessed/aryaman/processed_data.json'))
for article in vis_articles:
    if article['id'] not in embeddings_dict:
        # print(article['id'])
        embeddings = get_embedding(article['Abstract'])
        embeddings_dict[article['id']] = {
            'doc_id': article['id'],
            'title': article['Title'],
            'summary': article['Abstract'],
            'content': article['Abstract'],
            'embedding': embeddings,
        }
save_json(embeddings_dict, 'data/result/VisPub/embeddings_all.json')


In [ ]:
embeddings_dict = json.load(open('data/result/VisPub/embeddings_all.json'))
print(len(embeddings_dict), len(vis_articles))

In [ ]:
save_json(list(embeddings_dict.values()), 'data/result/VisPub/embeddings_all.json')

In [4]:
# VisPub entity embeddings
# vispub_entity_dict = json.load(open('data/result/VisPub/network/entities.json'))
# vispub_entity_dict = json.load(open('data/raw/VisPub/keywords_w_explanation.json'))
# entity_embeddings = json.load(open('data/result/VisPub/keyword_embeddings.json'))
entity_embeddings = {}
vispub_entity_dict = json.load(open('data/result/VisPub/entities_w_explanation.json'))
count = 0
length = len(vispub_entity_dict)
error_ids = []
for entity in vispub_entity_dict.values():
    count += 1
    print("{}/{}".format(count, length))
    if entity['id'] in entity_embeddings: continue
    # if count == 10: break
    try:
        # mentions = [mention['mention'] for mention in entity['mentions']]
        # mention_embeddings = [get_embedding(mention) for mention in mentions]
        # embedding = np.mean(mention_embeddings, axis=0).tolist()
        explanation = entity['explanation']
        embedding = get_embedding(explanation)
        entity_embeddings[entity['id']] = {
            "id": entity['id'],
            "title": entity['id'],
            "explanation": explanation,
            "embedding": embedding,
        }
        save_json(entity_embeddings, 'data/result/VisPub/entity_embeddings_w_explanation.json')
    except:
        error_ids.append(entity['id'])
        continue

1/1436
2/1436
3/1436
4/1436
5/1436
6/1436
7/1436
8/1436
9/1436
10/1436
11/1436
12/1436
13/1436
14/1436
15/1436
16/1436
17/1436
18/1436
19/1436
20/1436
21/1436
22/1436
23/1436
24/1436
25/1436
26/1436
27/1436


In [ ]:
save_json(entity_embeddings, 'data/result/VisPub/entities_embeddings.json')